In [2]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample

URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage=0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage=0.80)

In [3]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Hybrid import LinearZScoreNormalizedHybridRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

model = LinearZScoreNormalizedHybridRecommender
slim = SLIMElasticNetRecommender(URM_train)
rp3 = RP3betaRecommender(URM_train)
itemknn = ItemKNNCFRecommender(URM_train)
slim.fit(topK=8894, l1_ratio=0.05565733019999427, alpha=0.0012979360257937668)
rp3.fit(topK=101, alpha=0.3026342852596128, beta=0.058468783118329024)
itemknn.fit(topK=31, shrink=435, similarity='tversky', normalize=True,
            feature_weighting='BM25', tversky_alpha=0.17113169506422393, tversky_beta=0.5684024974085575)

from Evaluation.Evaluator import EvaluatorHoldout

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], ignore_users=[])

SLIMElasticNetRecommender: URM Detected 469 ( 3.7%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 343 ( 1.5%) items with no interactions.
RP3betaRecommender: URM Detected 469 ( 3.7%) users with no interactions.
RP3betaRecommender: URM Detected 343 ( 1.5%) items with no interactions.
ItemKNNCFRecommender: URM Detected 469 ( 3.7%) users with no interactions.
ItemKNNCFRecommender: URM Detected 343 ( 1.5%) items with no interactions.
SLIMElasticNetRecommender: Processed 22222 (100.0%) in 4.37 min. Items per second: 84.76
RP3betaRecommender: Similarity column 22222 (100.0%), 7405.99 column/sec. Elapsed time 3.00 sec
Similarity column 22222 (100.0%), 12263.80 column/sec. Elapsed time 1.81 sec
EvaluatorHoldout: Ignoring 2598 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [7]:
recommender = model(URM_train, [slim, rp3, itemknn])

SLIMElasticNetRP3betaItemKNNCFZScoreNormHybridRecommender: URM Detected 469 ( 3.7%) users with no interactions.
SLIMElasticNetRP3betaItemKNNCFZScoreNormHybridRecommender: URM Detected 343 ( 1.5%) items with no interactions.


In [8]:
import optuna as op


def objective(trial):
    alphas = [trial.suggest_float("alphas[0]", 0, 2), trial.suggest_float("alphas[1]", 0, 2),
              trial.suggest_float("alphas[2]", 0, 2)]

    #recommender = model(URM_train, [slim, rp3, itemknn])
    #recommender.fit(**trial.params)
    recommender.fit(alphas)

    result, _ = evaluator_validation.evaluateRecommender(recommender)
    MAP_result = result["MAP_MIN_DEN"].item()

    return MAP_result

In [10]:
best_params = {
    "alphas": [1, 1, 1],
}

study_name = "hybrid-zscore-study"  # Unique identifier of the study.
storage_name = "sqlite:///db.db"
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize", load_if_exists=True)
#study.enqueue_trial(best_params)
study.optimize(objective, n_trials=100)

[I 2023-11-30 18:44:10,600] Using an existing study with name 'hybrid-zscore-study' instead of creating a new one.


EvaluatorHoldout: Processed 10040 (100.0%) in 4.85 sec. Users per second: 2071


[I 2023-11-30 18:44:18,856] Trial 20 finished with value: 0.06294703567940371 and parameters: {'alphas[0]': 1.227094327261709, 'alphas[1]': 1.6145538280314196, 'alphas[2]': 1.0843079644964528}. Best is trial 10 with value: 0.06321814590341057.


EvaluatorHoldout: Processed 10040 (100.0%) in 4.83 sec. Users per second: 2078


[I 2023-11-30 18:44:26,993] Trial 21 finished with value: 0.06329357257603493 and parameters: {'alphas[0]': 1.6066169837209139, 'alphas[1]': 1.3237272247734135, 'alphas[2]': 0.6031199316600462}. Best is trial 21 with value: 0.06329357257603493.


...

[I 2023-11-30 19:41:22,091] Trial 117 finished with value: 0.06328933047414764 and parameters: {'alphas[0]': 1.7002826153576542, 'alphas[1]': 1.5971406030733333, 'alphas[2]': 0.44271323102158794}. Best is trial 70 with value: 0.06342829511821683.


EvaluatorHoldout: Processed 10040 (100.0%) in 4.80 sec. Users per second: 2091


[I 2023-11-30 19:41:30,206] Trial 118 finished with value: 0.06325674401474463 and parameters: {'alphas[0]': 1.7333472273622994, 'alphas[1]': 1.4492817050030014, 'alphas[2]': 0.4898722963901122}. Best is trial 70 with value: 0.06342829511821683.


EvaluatorHoldout: Processed 10040 (100.0%) in 4.77 sec. Users per second: 2105


[I 2023-11-30 19:41:38,293] Trial 119 finished with value: 0.06327030047484533 and parameters: {'alphas[0]': 1.6109830967444043, 'alphas[1]': 1.5449100346668938, 'alphas[2]': 0.6766364881269115}. Best is trial 70 with value: 0.06342829511821683.


In [11]:
study.best_params

{'alphas[0]': 1.849455433517727,
 'alphas[1]': 1.5825559104774185,
 'alphas[2]': 0.6146994012804048}

In [12]:
final = model(URM_train_validation, [slim, rp3, itemknn])
final.fit(alphas=[study.best_params["alphas[0]"], study.best_params["alphas[1]"], study.best_params["alphas[2]"]])
#final.fit(**study.best_params)

SLIMElasticNetRP3betaItemKNNCFZScoreNormHybridRecommender: URM Detected 227 ( 1.8%) users with no interactions.
SLIMElasticNetRP3betaItemKNNCFZScoreNormHybridRecommender: URM Detected 135 ( 0.6%) items with no interactions.


In [13]:
from Evaluation.Evaluator import EvaluatorHoldout

evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users=[])
evaluator_test.evaluateRecommender(final)

EvaluatorHoldout: Ignoring 2183 (17.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Processed 10455 (100.0%) in 4.93 sec. Users per second: 2122


(       PRECISION PRECISION_RECALL_MIN_DEN   RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                     
 10      0.087996                 0.149484  0.12505  0.045037    0.075668   
 
              MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                        ...   
 10      0.254339  0.138042  0.103301  0.491822      0.334334  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.827267          0.406868    0.827267       0.061158   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10           10.257095                   0.997729             0.175433   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                      

In [14]:
#final.fit(alphas=[3,1.25])
evaluator_test.evaluateRecommender(slim)

EvaluatorHoldout: Processed 10455 (100.0%) in 3.58 sec. Users per second: 2922


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL      MAP MAP_MIN_DEN  \
 cutoff                                                                     
 10      0.078785                 0.136544  0.115642  0.03788    0.065813   
 
              MRR      NDCG       F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                       ...   
 10      0.230194  0.123547  0.09372  0.471545      0.292971  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.827267          0.390093    0.827267       0.050084   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10           10.015474                   0.997357             0.145463   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                         